# Prérequis

Premièrement, nous avons besoin d'importer les bibliothèques python nécessaires pour notre algorithme.

La bibliothèque la plus importante est py_entitymatching, qui est développée par Anhais Group dans le cadre du projet Magellan.

https://sites.google.com/site/anhaidgroup/home

In [ ]:
# importation des bibliothèques requises (notamment py_entitymatching)

!pip install unidecode
!pip install fuzzywuzzy
!pip install missingno
!pip uninstall Cython -y
!pip uninstall py_entitymatching -y
!pip uninstall py_stringsimjoin -y
!pip install py_stringsimjoin==0.3.3
!pip install py_entitymatching==0.4.1


import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
from fuzzywuzzy import fuzz
from unidecode import unidecode
import missingno as msno
import py_stringsimjoin as ssj
import py_stringmatching as sm
import py_entitymatching as em
import math

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.1 MB/s eta 0:00:00
Found existing installation: Cython 3.0.8
Uninstalling Cython-3.0.8:
  Successfully uninstalled Cython-3.0.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for py_stringsimjoin: filename=py_stringsimjoin-0.3.3-cp310-cp310-linux_x86_64.whl size=4517599 sha256=635504cb976ee77db63f78e7d4041ea52594131375e068006907d671ae20ec16
  Stored in directory: /root/.cache/pip/wheels/63/47/e6/f221c81b7b7da486f12483e0e1e766e8be3b1c25e1f08a9203
  Created wheel for py_stringmatching: filename=py_stringmatching-0.4.5-cp310-cp310-linux_x86_64.whl size=2834020 sha256=e9a80b13d3a13f60d7411648d40d3f4187999f701bacf9c4389ab7a8103511f9
  Stored in directory: /root/.cache/pip/wheels/c2/8b/ee/728d0fd5aaf2230776060644dd45e2ef8

--------

# Training

Now that we know which feature is important. So we reduce the size of model for better training

In [ ]:
labeled = pd.read_csv('/content/dataset_annote_V3.csv')
labeled = labeled.dropna(subset=['id','IdentifiantImmeuble'])

labeled = labeled[~labeled['id'].duplicated()]
labeled = labeled[~labeled['IdentifiantImmeuble'].duplicated()]
labeled = labeled.fillna('')

labeled = labeled.astype({'id': 'string', 'numero' : 'float32' , 'rep' : 'string', 'TypeVoie': 'string', 'NomVoieReste': 'string', 'code_postal': 'int32',
                          'nom_commune': 'string', 'x': 'float32', 'y': 'float32', 'IdentifiantImmeuble': 'string', 'NumeroVoieImmeuble': 'float32',
                          'ComplementNumeroVoieImmeuble': 'string', 'TypeVoieImmeuble': 'string', 'NomVoieImmeuble': 'string',
                          'CodePostalImmeuble': 'int32', 'CommuneImmeuble': 'string', 'CoordonneeImmeubleX': 'float32',
                          'CoordonneeImmeubleY': 'float32', 'Label': 'int32'})

A = labeled.iloc[:, :len(labeled.columns)//2]
B = labeled.iloc[:, len(labeled.columns)//2:-1]
B['TypeVoieImmeuble'] = B['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')

def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['rep'] = A['rep'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)
ban = A.copy()
ipe = B.copy()
labeled = pd.concat([ban,ipe,labeled.iloc[:,-1]],axis=1)
nom_colonnes = {'id': 'ltable_id','numero' : 'ltable_NumeroVoieImmeuble','rep' : 'ltable_ComplementNumeroVoieImmeuble','TypeVoie' : 'ltable_TypeVoieImmeuble','NomVoieReste' : 'ltable_NomVoieImmeuble','code_postal' : 'ltable_CodePostalImmeuble','nom_commune' : 'ltable_CommuneImmeuble','x' : 'ltable_CoordonneeImmeubleX','y' : 'ltable_CoordonneeImmeubleY', 'IdentifiantImmeuble' : 'rtable_id', 'NumeroVoieImmeuble' : 'rtable_NumeroVoieImmeuble', 'ComplementNumeroVoieImmeuble' : 'rtable_ComplementNumeroVoieImmeuble', 'TypeVoieImmeuble' : 'rtable_TypeVoieImmeuble', 'NomVoieImmeuble' : 'rtable_NomVoieImmeuble', 'CodePostalImmeuble' : 'rtable_CodePostalImmeuble', 'CommuneImmeuble': 'rtable_CommuneImmeuble', 'CoordonneeImmeubleX' : 'rtable_CoordonneeImmeubleX', 'CoordonneImmeubleY' : 'rtable_CoordonneeImmeubleY', 'Label' : 'label'}
labeled = labeled.rename(columns = nom_colonnes)

labeled['ltable_TypeVoieImmeuble'] = labeled['ltable_TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')
nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
A = A.rename(columns=nom_colonnes)
labeled['_id'] = range(len(labeled))

#We need to set key for each table to create metadata so that we can continue the next step
em.set_key(labeled,'_id')
em.set_fk_ltable(labeled, 'ltable_id')
em.set_fk_rtable(labeled, 'rtable_id')
em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')
em.set_ltable(labeled, A)
em.set_rtable(labeled, B)

G = em.sample_table(labeled, labeled.shape[0])

#Customize the calculation of distance between 2 tuples
def real_distance(ltuple,rtuple):
  return np.sqrt((ltuple['CoordonneeImmeubleX']-rtuple['CoordonneeImmeubleX'])**2+(ltuple['CoordonneeImmeubleY']-rtuple['CoordonneeImmeubleY'])**2)

match_f = em.get_features_for_matching(A,B, validate_inferred_attr_types = False)

em.add_blackbox_feature(match_f,'real_distance',real_distance)
feature_to_remain = [6,13,23,26,38,54]
match_f.drop([i for i in range(match_f.shape[0]) if i not in feature_to_remain],inplace=True)
match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
6,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble,NumeroVoieImmeuble,None,None,exact_match,<function NumeroVoieImmeuble_NumeroVoieImmeuble_exm at 0x7888de6cf400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
13,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,ComplementNumeroVoieImmeuble,ComplementNumeroVoieImmeuble,None,None,monge_elkan,<function ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel at 0x7888de6cd750>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
23,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,TypeVoieImmeuble,TypeVoieImmeuble,None,None,lev_sim,<function TypeVoieImmeuble_TypeVoieImmeuble_lev_sim at 0x7888de6cd6c0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
26,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble,NomVoieImmeuble,qgm_3,qgm_3,jaccard,<function NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3 at 0x7888de6cd630>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
38,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble,CommuneImmeuble,qgm_3,qgm_3,jaccard,<function CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3 at 0x7888de6cd2d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
54,real_distance,None,None,None,None,None,<function real_distance at 0x7888de6cd480>,None,False


In [ ]:
H_train = em.extract_feature_vecs(G, feature_table=match_f , attrs_after=['label'])
H_train

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance,label
0,0,12065_0062_00035,IMB/12065/X/0087,1,0.0,0.333333,0.642857,1.00,0.000000,0
1,1,12065_0062_00036,IMB/12065/X/0088,1,1.0,0.800000,0.529412,1.00,0.000000,0
2,2,12065_0062_00031,IMB/12065/X/0083,1,1.0,0.800000,0.461538,1.00,412.310563,0
3,3,12065_0062_00033,IMB/12065/X/0085,1,1.0,0.142857,0.363636,1.00,100498.756211,0
4,4,12140_b267_03121,IMB/12208/X/003V,0,1.0,0.166667,0.142857,0.00,95232.252832,0
...,...,...,...,...,...,...,...,...,...,...
963,930,12168_0067_00037,IMB/12118/X/002T,0,1.0,0.200000,0.074074,0.00,100019.943861,0
964,931,12168_0067_00039,IMB/12057/X/0060,0,1.0,0.600000,0.035714,0.00,64431.636649,0
965,932,12168_0067_00020,IMB/12258/X/00AQ,0,1.0,0.000000,0.037037,0.00,93437.572856,0
966,933,12168_ip2tbt_00009,IMB/12064/X/004S,0,1.0,0.000000,0.000000,0.00,72727.969602,0


In [ ]:
H_train = em.impute_table(H_train, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], strategy='mean')

In [ ]:
dt = em.RFMatcher(n_estimators=300, max_depth = 15)

In [ ]:
train_test = em.split_train_test(H_train, train_proportion=0.7)
devel_set = train_test['train']
eval_set = train_test['test']

In [ ]:
dt.fit(table=devel_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label')


In [ ]:
pred_table = dt.predict(table=eval_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id','label'], target_attr='predicted_labels', append=True, inplace=True)

In [ ]:
pred_table

,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance,label,predicted_labels
853,832,12225_0210_00008,IMB/12159/X/004N,0.0,1.0,1.000000,0.166667,0.000000,78797.198476,0.0,0.0
744,725,12061_07jucp_00002,IMB/12062/X/005J,0.0,1.0,0.166667,0.333333,0.314286,41906.904671,0.0,0.0
773,753,12208_0391_00523,IMB/12048/X/000R,0.0,1.0,0.000000,0.000000,0.000000,83090.265167,0.0,0.0
741,722,12044_0290_00008,IMB/12277/X/00A2,0.0,1.0,0.166667,0.000000,0.000000,112237.445588,0.0,0.0
473,467,12161_hos2m7_01075,IMB/12102/X/00QQ,0.0,1.0,0.400000,0.000000,0.000000,28179.883900,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
253,247,12089_0870_00018_bis,IMB/12089/X/01D4,1.0,1.0,0.666667,1.000000,1.000000,32.078091,1.0,1.0
494,487,12298_0049_00001,IMB/12228/X/008L,1.0,1.0,0.666667,1.000000,0.000000,63722.446048,0.0,0.0
786,766,12062_bkl7xu_00376,IMB/12094/X/00FQ,0.0,1.0,0.142857,0.157895,0.000000,62746.990710,0.0,0.0
185,181,12148_nmjlxe_00028,IMB/12148/X/006W,1.0,1.0,1.000000,1.000000,1.000000,12.976541,1.0,1.0


In [ ]:
pred_table[pred_table['label']==1.0]

,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance,label,predicted_labels
135,134,12012_j7y8bu_00006,IMB/12012/X/007F,1.0,1.0,0.909091,1.000000,1.0,0.000000,1.0,1.0
307,301,12068_0290_00118,IMB/12068/X/001M,1.0,1.0,1.000000,1.000000,1.0,12.720708,1.0,1.0
278,272,12013_0565_00012,IMB/12013/X/00AO,1.0,1.0,1.000000,0.684211,1.0,12.976541,1.0,1.0
709,690,12120_0288_00320,IMB/12120/X/00QQ,1.0,1.0,1.000000,1.000000,1.0,12.838084,1.0,1.0
568,560,12131_0525_00148,IMB/12131/X/00O7,1.0,1.0,0.857143,1.000000,1.0,22.102036,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
83,82,12300_0030_00015,IMB/12300/X/02JH,1.0,1.0,1.000000,0.684211,1.0,12.992786,1.0,1.0
380,374,12076_nwcw6k_00001,IMB/12076/X/00RG,1.0,1.0,1.000000,1.000000,1.0,12.880612,1.0,1.0
404,398,12065_0048_02069,IMB/12065/X/008T,1.0,1.0,1.000000,1.000000,1.0,12.884705,1.0,1.0
253,247,12089_0870_00018_bis,IMB/12089/X/01D4,1.0,1.0,0.666667,1.000000,1.0,32.078091,1.0,1.0


In [ ]:
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

OrderedDict([('prec_numerator', 124.0),
             ('prec_denominator', 126.0),
             ('precision', 0.9841269841269841),
             ('recall_numerator', 124.0),
             ('recall_denominator', 124.0),
             ('recall', 1.0),
             ('f1', 0.9919999999999999),
             ('pred_pos_num', 126.0),
             ('false_pos_num', 2.0),
             ('false_pos_ls',
              [('12065_0062_00030', 'IMB/12065/X/0082'),
               ('12065_0062_00034', 'IMB/12065/X/0086')]),
             ('pred_neg_num', 155.0),
             ('false_neg_num', 0.0),
             ('false_neg_ls', [])])

# Prediction

Dans cette partie, nous utilisons le modèle entrainé sur le dataset annoté d'entrainement afin de prédire si les adresses sont similaires ou non.

In [ ]:
#si vous avez enregistré les dataframes de la section précédente sous forme de CSV pour les réutiliser, vous pouvez les charger ici en précisant le chemin des CSV.
ban_reduced = pd.read_csv('/content/ban_reduced.csv')
ipe_reduced = pd.read_csv('/content/ipe_reduced.csv')

In [ ]:
A = ipe_reduced.copy()
B = ban_reduced.copy()

#A.rename(columns={'IdentifiantImmeuble': 'id'}, inplace=True)
#B.rename(columns={'IdentifiantImmeuble': 'id'}, inplace=True)

em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')


shape_A = A.shape
shape_B = B.shape
print(shape_A)
print(shape_B)

(283831, 9)
(152683, 9)


In [ ]:
def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['ComplementNumeroVoieImmeuble'] = A['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)


### Blocking

Comme pour l'algorithme 1 basé sur nos propres règles, nous définissons des colonnes pour lesquelles la correspondance doit être stricte entre les 2 adresses :
- NumeroVoieImmeuble
- ComplementNumeroVoieImmeuble
- TypeVoieImmeuble
- CodePostalImmeuble


In [ ]:
# define the blocker attributes to check and put in 1 column (because AttrEquivalenceBlocker can only handle 1 column comparison)
# for the unknown NumeroVoieImmeuble, we impute with the value 0
'''
A['NumeroVoieImmeuble'] = A['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(A[A['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the IPE dataset')

B['NumeroVoieImmeuble'] = B['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(B[B['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the BAN dataset')

A['CodePostalImmeuble'] = A['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(A[A['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the IPE dataset')

B['CodePostalImmeuble'] = B['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(B[B['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the BAN dataset')
'''



A['blocking_data'] = A['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  A['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + A['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + A['CodePostalImmeuble'].fillna('').astype(str)
B['blocking_data'] = B['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  B['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + B['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + B['CodePostalImmeuble'].fillna('').astype(str)



In [ ]:
attributes = ['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']

ab = em.AttrEquivalenceBlocker()

# Use block_tables to apply blocking over two input tables.
C = ab.block_tables(A, B,
                    l_block_attr='blocking_data', r_block_attr='blocking_data',
                    l_output_attrs=attributes,
                    r_output_attrs=attributes,
                    l_output_prefix='ltable_', r_output_prefix='rtable_')

C

,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,rtable_CoordonneeImmeubleY
0,0,IMB/12001/X/0001,12001_0052_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,Adrien Vezinhet,12630,Agen-d'Aveyron,673832.68,6362135.56
1,1,IMB/12001/X/0001,12001_0040_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,Daude de Prades,12630,Agen-d'Aveyron,673699.44,6362045.91
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82
3,3,IMB/12001/X/0001,12001_0067_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52
4,4,IMB/12001/X/0001,12001_0069_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Croux,12630,Agen-d'Aveyron,674318.84,6362009.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.7426,6344054.694,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.2682,6344332.362,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.5716,6344372.058,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.0282,6347025.643,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43


### ML prediction

In [ ]:
H = em.extract_feature_vecs(C, feature_table=match_f)
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble'], strategy='mean')
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:05:35


,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance
0,0,IMB/12001/X/0001,12001_0052_00004,1.0,1.0,1.0,0.000000,1.0,869.844563
1,1,IMB/12001/X/0001,12001_0040_00004,1.0,1.0,1.0,0.028571,1.0,912.011620
2,2,IMB/12001/X/0001,12001_0083_00004,1.0,1.0,1.0,1.000000,1.0,33.955654
3,3,IMB/12001/X/0001,12001_0067_00004,1.0,1.0,1.0,0.250000,1.0,2957.680514
4,4,IMB/12001/X/0001,12001_0069_00004,1.0,1.0,1.0,0.222222,1.0,512.126658
...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,1.0,1.0,1.0,1.000000,1.0,40.512861
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,1.0,1.0,1.0,1.000000,1.0,44.155615
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,1.0,1.0,1.0,1.000000,1.0,47.087184
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,1.0,1.0,1.0,1.000000,1.0,18.890861


In [ ]:
print(devel_set.columns)
print(H.columns)

Index(['_id', 'ltable_id', 'rtable_id',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_exm',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel',
       'TypeVoieImmeuble_TypeVoieImmeuble_lev_sim',
       'NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3',
       'CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3', 'real_distance',
       'label'],
      dtype='object')
Index(['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_exm',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel',
       'TypeVoieImmeuble_TypeVoieImmeuble_lev_sim',
       'NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3',
       'CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3', 'real_distance'],
      dtype='object')


C'est dans cette cellule que nous utilisons notre modèle de DecisionTree  afin d'évaluer si 2 adresses sont similaires ou non.
Si les adresses sont similaires, predicted_labels = 1, et 0 sinon.
Le désavantage de cette méthode est que la prédiciton est binaire, et donc contrairement au premier algorithme proposé, on n'obtient pas réellement un % de confiance de la prédiction, même si nous pouvons le recalculer grâce aux mêmes méthodes que dans le 1er algorithme (Levensthein distance).

En réalité, cet algorithme de machine learning se base aussi sur des calculs de distance entre les différentes composantes de l'adresse, mais il utilise un processus d'arbre de décision afin d'optimiser la sélection.

In [ ]:
#dt.fit(table=devel_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label')
#pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble','predicted_labels'], target_attr='predicted_labels', append=True, inplace=True)
pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble'], target_attr='predicted_labels', append=True, inplace=True)

pred_table


,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance,predicted_labels
0,0,IMB/12001/X/0001,12001_0052_00004,1.0,1.0,1.0,0.000000,1.0,869.844563,0.0
1,1,IMB/12001/X/0001,12001_0040_00004,1.0,1.0,1.0,0.028571,1.0,912.011620,0.0
2,2,IMB/12001/X/0001,12001_0083_00004,1.0,1.0,1.0,1.000000,1.0,33.955654,1.0
3,3,IMB/12001/X/0001,12001_0067_00004,1.0,1.0,1.0,0.250000,1.0,2957.680514,0.0
4,4,IMB/12001/X/0001,12001_0069_00004,1.0,1.0,1.0,0.222222,1.0,512.126658,0.0
...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,1.0,1.0,1.0,1.000000,1.0,40.512861,1.0
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,1.0,1.0,1.0,1.000000,1.0,44.155615,1.0
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,1.0,1.0,1.0,1.000000,1.0,47.087184,1.0
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,1.0,1.0,1.0,1.000000,1.0,18.890861,1.0


In [ ]:
print('We found', len(pred_table[pred_table['predicted_labels'] == 1]), 'matching adresses')

We found 79697 matching adresses


In [ ]:
pred_table[pred_table['predicted_labels'] == 1].to_csv('matched_address.csv')

In [ ]:
importances = dt.clf.feature_importances_
# Get indices of features sorted by importance
indices = np.argsort(importances)[::-1]

# Print feature ranking
print("Feature ranking:")
for f in range(6):
    print("%d. feature %str (%f)" % (f + 1, devel_set.columns[3:-1][indices[f]], importances[indices[f]]))

Feature ranking:
1. feature NumeroVoieImmeuble_NumeroVoieImmeuble_exmtr (0.283773)
2. feature real_distancetr (0.258313)
3. feature NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3tr (0.244327)
4. feature CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3tr (0.142980)
5. feature TypeVoieImmeuble_TypeVoieImmeuble_lev_simtr (0.066221)
6. feature ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_meltr (0.004386)


If there are 2 or more addresses in left table that have match to one address in right table (or vice versa). We take the match that have lowest distance in coordinate.

In [ ]:
devel_set.columns

Index(['_id', 'ltable_id', 'rtable_id',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_exm',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel',
       'TypeVoieImmeuble_TypeVoieImmeuble_lev_sim',
       'NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3',
       'CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3', 'real_distance',
       'label'],
      dtype='object')

In [ ]:
pred_table[pred_table['predicted_labels'] == 1].to_csv('test.csv')

In [ ]:
match_table = pred_table[pred_table['predicted_labels'] == 1]
# Sort the DataFrame by column 'B' in ascending order
df_sorted = match_table.sort_values(by='real_distance')

# Keep only the first occurrence of each value in column 'A' (which has the minimum value in column 'B')
match_table1 = df_sorted.drop_duplicates(subset='ltable_IdentifiantImmeuble', keep='first')
match_table2 = match_table1.drop_duplicates(subset='rtable_IdentifiantImmeuble', keep='first')
print(len(match_table2))
match_table2.to_csv('matched_address_V4.csv')

72886
